In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import ipywidgets as pyw
import hyperspy.api as hys
import tkinter.filedialog as tkf
from tabulate import tabulate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import linalg as LA
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid

In [ ]:
# create a customized colorbar
color_rep = ["black", "red", "green", "blue", "orange", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))
custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

def zero_one_rescale(spectrum):
    """
    normalize one spectrum from 0.0 to 1.0
    """
    spectrum = spectrum.clip(min=0.0)
    min_val = np.min(spectrum)
    
    rescaled = spectrum - min_val
    
    if np.max(rescaled) != 0:
        rescaled = rescaled / np.max(rescaled)
    
    return rescaled

def indices_at_r(shape, radius, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    r = np.hypot(y - center[0], x - center[1])
    r = np.around(r)
    
    ri = np.where(r == radius)
    
    angle_arr = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            angle_arr[i, j] = np.angle(complex(x[i, j]-center[1], y[i, j]-center[0]), deg=True)
            
    angle_arr = angle_arr + 180
    angle_arr = np.around(angle_arr)
    
    ai = np.argsort(angle_arr[ri])
    r_sort = (ri[1][ai], ri[0][ai])
    a_sort = np.sort(angle_arr[ri])
        
    return r_sort, a_sort

def circle_flatten(f_stack, radial_range, c_pos):
    k_indx = []
    k_indy = []
    
    for r in range(radial_range[0], radial_range[1], radial_range[2]):
        tmp_k, tmp_a = indices_at_r(f_stack.shape[2:], r, c_pos)
        k_indx.extend(tmp_k[0].tolist())
        k_indy.extend(tmp_k[1].tolist())
    
    k_indx = np.asarray(k_indx)
    k_indy = np.asarray(k_indy)
    flat_data = f_stack[:, :, k_indy, k_indx]
    
    return flat_data

def flattening(fdata, flat_option="box", crop_dist=None, c_pos=None):
    
    fdata_shape = fdata.shape
    if flat_option == "box":
        if crop_dist:     
            box_size = np.array([crop_dist, crop_dist])
        
            for i in range(num_img):
                h_si = np.floor(c_pos[0]-box_size[0]).astype(int)
                h_fi = np.ceil(c_pos[0]+box_size[0]).astype(int)
                w_si = np.floor(c_pos[1]-box_size[1]).astype(int)
                w_fi = np.ceil(c_pos[1]+box_size[1]).astype(int)

            tmp = fdata[:, :, h_si:h_fi, w_si:w_fi]
            
            fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            ax.imshow(np.log(np.mean(tmp, axis=(0, 1))), cmap="viridis")
            ax.axis("off")
            plt.show()
            
            tmp = tmp.reshape(fdata_shape[0], fdata_shape[1], -1)
            return tmp

        else:
            tmp = fdata.reshape(fdata_shape[0], fdata_shape[1], -1)
            return tmp

        
    elif flat_option == "radial":
        if len(crop_dist) != 3:
            print("Warning! 'crop_dist' must be a list containing 3 elements")
            
        tmp = circle_flatten(fdata, crop_dist, c_pos)
        return tmp
        
    else:
        print("Warning! Wrong option ('flat_option')")
        return

def fourd_roll_axis(stack):
    stack = np.rollaxis(np.rollaxis(stack, 2, 0), 3, 1)
    return stack

def reshape_coeff(coeffs, new_shape):
    """
    reshape a coefficient matrix to restore the original scanning shapes.
    """
    coeff_reshape = []
    for i in range(len(new_shape)):
        temp = coeffs[:int(new_shape[i, 0]*new_shape[i, 1]), :]
        coeffs = np.delete(coeffs, range(int(new_shape[i, 0]*new_shape[i, 1])), axis=0)
        temp = np.reshape(temp, (new_shape[i, 0], new_shape[i, 1], -1))
        #print(temp.shape)
        coeff_reshape.append(temp)
        
    return coeff_reshape

def radial_indices(shape, radial_range, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    r = np.hypot(y - center[0], x - center[1])
    ri = np.ones(r.shape)
    
    if len(np.unique(radial_range)) > 1:
        ri[np.where(r <= radial_range[0])] = 0
        ri[np.where(r > radial_range[1])] = 0
        
    else:
        r = np.round(r)
        ri[np.where(r != round(radial_range[0]))] = 0
    
    return ri

In [ ]:
file_adr = []

In [ ]:
file_adr.extend(tkf.askopenfilenames())
print(len(file_adr))
print(*file_adr, sep="\n")

In [ ]:
num_img = len(file_adr)
print(num_img)

In [ ]:
# load 4D-STEM data
data_original = []
data_shape = []
for i in range(num_img):
    tmp = tifffile.imread(file_adr[i])
    print(tmp.shape)
    data_shape.append(list(tmp.shape[:2]))
    data_original.append(tmp)
    
data_shape = np.asarray(data_shape)

In [ ]:
# find the center position
center_pos = []
cbox_edge = 15
center_removed_ = False
for i in range(num_img):
    mean_dp = np.mean(data_original[i], axis=(0, 1))
    cbox_outy = int(mean_dp.shape[0]/2 - cbox_edge/2)
    cbox_outx = int(mean_dp.shape[1]/2 - cbox_edge/2)
    center_box = mean_dp[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
    Y, X = np.indices(center_box.shape)
    com_y = np.sum(center_box * Y) / np.sum(center_box)
    com_x = np.sum(center_box * X) / np.sum(center_box)
    c_pos = [np.around(com_y+cbox_outy), np.around(com_x+cbox_outx)]
    center_pos.append(c_pos)
print(center_pos)

In [ ]:
np.seterr(divide='ignore')
for i in range(num_img):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(np.log(np.mean(data_original[i], axis=(0, 1))), cmap="viridis")
    ax.scatter(center_pos[i][1], center_pos[i][0], c="r", s=10)
    ax.axis("off")
    plt.show()

In [ ]:
# get rid of the center beam (optional)
center_removed_ = True
center_radius = 10
data_cr = []
for i in range(num_img):
    ri = radial_indices(data_original[i].shape[2:], [center_radius, 100], center=center_pos[i])
    data_cr.append(np.multiply(data_original[i], ri))

In [ ]:
if center_removed_:
    for i in range(num_img):
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.imshow(np.log(np.mean(data_cr[i], axis=(0, 1))), cmap="viridis")
        ax.scatter(center_pos[i][1], center_pos[i][0], c="r", s=10)
        ax.axis("off")
        plt.show()

In [ ]:
# 2D diffraction pattern -> 1D data
# option 1 : flatten a box
radial_flat_ = False

dataset = []
half_size = 30
for i in range(num_img):
    if center_removed_:
        flattened = flattening(data_cr[i], flat_option="box", crop_dist=30, c_pos=center_pos[i])
    
    else:
        flattened = flattening(data_original[i], flat_option="box", crop_dist=30, c_pos=center_pos[i])
    
    dataset.append(flattened)
    
s_length = (30*2)**2

In [ ]:
# 2D diffraction pattern -> 1D data
# option 2 : flatten radially
radial_flat_ = True

dataset = []
radial_range = [10, 30, 1]
k_indx = []
k_indy = []
a_ind = []

for r in range(radial_range[0], radial_range[1], radial_range[2]):
    tmp_k, tmp_a = indices_at_r((radial_range[1]*2, radial_range[1]*2), r, (radial_range[1], radial_range[1]))
    k_indx.extend(tmp_k[0].tolist())
    k_indy.extend(tmp_k[1].tolist())
    a_ind.extend(tmp_a.tolist())
    
s_length = len(k_indx)

for i in range(num_img):
    if center_removed_:
        flattened = circle_flatten(data_cr[i], radial_range, center_pos[i])
    else:
        flattened = circle_flatten(data_original[i], radial_range, center_pos[i])
        
    dataset.append(flattened)

In [ ]:
# create the input dataset
dataset_flat = []
for i in range(num_img):
    print(dataset[i].shape)
    dataset_flat.extend(dataset[i].reshape(-1, s_length))
dataset_flat = np.asarray(dataset_flat).clip(min=0.0)
print(dataset_flat.shape)

In [ ]:
# convert values into log scale (optional)
dataset_flat[np.where(dataset_flat==0.0)] = 1.0
dataset_flat = np.log(dataset_flat)

In [ ]:
# max-normalize each flattened diffraction pattern (optional)
dataset_flat = dataset_flat / np.max(dataset_flat, axis=1)[:, np.newaxis]
dataset_flat = dataset_flat.clip(min=0.0)
print(dataset_flat.shape)

In [ ]:
total_num = len(dataset_flat)
ri = np.random.choice(total_num, total_num, replace=False)

dataset_input = dataset_flat[ri]

In [ ]:
class simple_VAE(nn.Module):
    def __init__(self, in_dim, h_dim, z_dim):
        super(simple_VAE, self).__init__()
        
        self.in_dim = in_dim
        self.z_dim = z_dim
        self.h_dim = h_dim
        
        enc_net = []
        enc_net.append(nn.Linear(self.in_dim, self.h_dim[0]))
        enc_net.append(nn.LeakyReLU(0.1))
        for i in range(1, len(h_dim)):
            enc_net.append(nn.Linear(self.h_dim[i-1], self.h_dim[i]))
            enc_net.append(nn.LeakyReLU(0.1))
        enc_net.append(nn.Linear(self.h_dim[-1], 2*z_dim))
        
        self.encoder = nn.Sequential(*enc_net)
        
        dec_net = []
        dec_net.append(nn.Linear(z_dim, self.h_dim[-1]))
        dec_net.append(nn.LeakyReLU(0.1))
        for i in range(len(h_dim), 1, -1):
            dec_net.append(nn.Linear(self.h_dim[i-1], self.h_dim[i-2]))
            dec_net.append(nn.LeakyReLU(0.1))
        dec_net.append(nn.Linear(self.h_dim[0], self.in_dim))
        dec_net.append(nn.Sigmoid())
        
        self.decoder = nn.Sequential(*dec_net)
        
    def encode(self, x):
        
        latent = self.encoder(x)
        mu = latent[:, :self.z_dim]
        logvar = latent[:, self.z_dim:]
        
        return mu, logvar
        
                
    def reparametrization(self, mu, logvar):
        
        return mu+torch.exp(0.5*logvar)*torch.randn_like(logvar)
        
        
    def decode(self, z):
        
        return self.decoder(z)
        
        
    def forward(self, x):
        
        mu, logvar = self.encode(x)
        z = self.reparametrization(mu, logvar)
        
        return mu, logvar, z, self.decode(z)

In [ ]:
if torch.cuda.is_available():
    print("%d gpu available"%(torch.cuda.device_count()))
    cuda_device = torch.device("cuda:0")
    print(torch.cuda.get_device_name(cuda_device))
    torch.cuda.set_device(cuda_device)
else:
    cuda_device = None

In [ ]:
h_dim = [1000, 1000]
z_dim = 3

parallel_ = True

model = simple_VAE(s_length, h_dim, z_dim)

if parallel_:
    model = nn.DataParallel(model)

model.cuda(cuda_device)
print(model)

In [ ]:
batch_size = 1000
mini_batches = [dataset_input[k:k+batch_size] for k in range(0, len(dataset_input), batch_size)]
print(len(mini_batches))

In [ ]:
n_epoch = 1000
optimizer = optim.Adam(model.parameters())
print(optimizer)

In [ ]:
start = time.time()
latent_z = []
model.train()
n_fig = 5
for epoch in range(n_epoch):
    loss_epoch = 0
    recon_loss = 0
    KLD_loss = 0
    for i, m_batch in enumerate(mini_batches):
        
        x = torch.from_numpy(mini_batches[i])
        x = x.to(torch.float32)
        x = x.to(cuda_device)
        x.requires_grad_(requires_grad=False)
        
        mu, logvar, z, x_ = model(x)
        
        reconstruction_error = F.mse_loss(x_, x, reduction="sum")
        KL_divergence = -0.5*torch.sum(1+logvar-mu**2-logvar.exp())
        
        loss = reconstruction_error + KL_divergence
        loss_epoch += loss.item()
        recon_loss += reconstruction_error.item()
        KLD_loss += KL_divergence.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch == n_epoch-1:
            z_batch = z.data.cpu().numpy().tolist()
            latent_z.extend(z_batch)   
            
    if epoch == 0:
        print(torch.cuda.memory_summary(device=cuda_device))
        
    if (epoch+1) % int(n_epoch/10) == 0:
        print(tabulate([
                        ["epoch", epoch+1], 
                        ["total loss", loss_epoch/total_num],
                        ["reconstruction error", recon_loss/total_num],
                        ["KL divergence", KLD_loss/total_num],
                        ["error ratio", reconstruction_error/KL_divergence],
                        ]))
        print("%.2f minutes have passed"%((time.time()-start)/60))
        
        fig, ax = plt.subplots(2, n_fig, figsize=(5*n_fig, 5))
        for i in range(n_fig):
            ax[0][i].imshow(x[i].data.cpu().reshape(cropped_size))
            ax[1][i].imshow(x_[i].data.cpu().reshape(cropped_size))
        fig.tight_layout()
        plt.show()

print("The training has been finished.")

In [ ]:
ae_coeffs = np.asarray(latent_z)
print(ae_coeffs.shape)

# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(ae_coeffs)
coeffs[ri] = ae_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape)

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib qt

In [ ]:
# visualize coefficient maps

if num_img != 1:
    for i in range(z_dim):
        fig, ax = plt.subplots(1, num_img, figsize=(7*num_img, 7))
        for j in range(num_img):
            tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], cmap="afmhot")
            ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
            ax[j].axis("off")
            fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()
else:            
    for i in range(z_dim):
        fig, ax = plt.subplots(1, 1, figsize=(7, 7*num_img))
        tmp = ax.imshow(coeffs_reshape[0][:, :, i], cmap="afmhot")
        ax.set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
        fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
        plt.show()

In [ ]:
# 2D subspace
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(7, 7))

def projection(c1, c2):
    ax.cla()
    ax.scatter(coeffs[:, c1], coeffs[:, c2], s=30, c="black", alpha=0.5)
    ax.grid()
    ax.set_xlabel("loading vector %d"%(c1+1), fontsize=15)
    ax.set_ylabel("loading vector %d"%(c2+1), fontsize=15)
    ax.tick_params(axis="both", labelsize=15)
    fig.canvas.draw()
    fig.tight_layout()

x_widget = pyw.IntSlider(min=0, max=z_dim-1, step=1, value=1)
y_widget = pyw.IntSlider(min=0, max=z_dim-1, step=1, value=2)

pyw.interact(projection, c1=x_widget, c2=y_widget)
plt.show()